# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [55]:
# Write your code below.

%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [56]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [63]:
import os
from glob import glob

# Write your code below.

# the parquet files from the 02_data engineering.ipynb is not in the PRICE_DATA directory but in Price_DATA
# that is why I'm not using PRICE_DATA
Price_DATA = os.getenv("PRICE_DATA")
print(Price_DATA)  #  just to see the result
# find all directories with a parquet files
parquet_files = glob(os.path.join(Price_DATA, "**/*.parquet"), recursive = True)

#dd_price = dd.read_parquet(parquet_files).set_index("ticker")
# Was > [] ; Now > ['../../05_src/data/temp\\parquet\\part.0.parquet',

#print(parquet_files)

#dd_price = dd.read_parquet(parquet_files).set_index("ticker")

../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [66]:
# Write your code below.
dd_price = dd.read_parquet(parquet_files).set_index("ticker")

parquet_dir = os.path.join(Price_DATA, "parquet")

print(dd_price .head() )

# extra code from data engineering notebook
#shutil.rmtree(parquet_dir, ignore_errors=True)
#os.makedirs(parquet_dir, exist_ok=True)

# Close Lag
# https://www.youtube.com/watch?v=19SBiW2h888 > 1:15:25;

# dd_feat

dd_feat = dd_price.groupby('ticker', group_keys=False).apply(
    # anonymous function
    lambda x: x.assign(
        # Close Lag
        Close_lag_1 = x['Close'].shift(1),
        # Adj_Close
        Adj_Close = x['Close'] / x['Close'].shift(1),
        # hi_lo_range
        hi_lo_range = x["High"] - x['Low'],
        # Calculate the return
        ticker_return = x['Close'] / x['Close'].shift(1) -1
    )
)





             Date  Open  High   Low  Close  Adj Close    Volume   source  Year
ticker                                                                        
AAU    2005-12-20  1.62  1.64  1.60   1.63       1.63   14100.0  AAU.csv  2005
AAU    2019-05-09  0.48  0.48  0.46   0.46       0.46  291800.0  AAU.csv  2019
AAU    2019-05-10  0.47  0.47  0.46   0.47       0.47  126600.0  AAU.csv  2019
AAU    2019-05-13  0.47  0.48  0.45   0.45       0.45  176800.0  AAU.csv  2019
AAU    2017-01-03  0.98  1.09  0.97   1.01       1.01  294600.0  AAU.csv  2017


C:\Users\Work\AppData\Local\Temp\ipykernel_18504\1430380639.py:17: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_price.groupby('ticker', group_keys=False).apply(


In [59]:
# # DIagnostics 

# # Write your code below.
# dd_price = dd.read_parquet(parquet_files).set_index("ticker")

# parquet_dir = os.path.join(Price_DATA, "parquet")

# # print(parquet_dir) # ../../05_src/data/temp/parquet

# print(dd_price .head() )

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [67]:
# Write your code below.

# run the calculations b/c lazy exe > and convert it to a PD DF > and storing it in a separate variable
Pd_dd_feat = dd_feat.compute()

# to convert to DF > .transform or .apply
#Pd_dd_feat = dd_feat.transform()


In [ ]:
# separating so the compute() does not have to be re run each time

# add a rolling avg (10) 
# cleaner to use .apply > Office hrs
Pd_dd_feat = Pd_dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        rolling_avg = x['ticker_return'].rolling(10).mean()
    ))

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

> It was not necessary to convert to a pandas dataframe. In fact the solution I used was to put the rolling average into the "dd_feat" dataframe after the fact because it was easier to do all the calculations at once.

+ Would it have been better to do it in Dask? Why?

> Performing the operations in Dask is better because I was able to "centralize" all my calculations into a single .assign() function. This would help my code to be more centralized and easier to maintain.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.